In [1]:
import xlrd
import re

In [2]:
from os import listdir
from os.path import isfile, join
from datetime import time
import datetime

In [3]:
path = "Data/"
file_names = [f for f in listdir(path) if isfile(join(path, f)) and (f.startswith('T_') or f.startswith('T-') )]

## Write Meal Time

In [4]:
eating_string = ['lunch', 'suppl', 'dinner', 'ipen', 'breakfast', 'b-fast','hpen','h-pen']
#non_eating_string = ['fasting', 'bg', 'exer']
mealtime_file = open("meal_time.csv", "w")

In [5]:
for file_name in file_names:
    file = xlrd.open_workbook('Data/'+ file_name) 
    id = re.match('T[_,-][0-9]+', file_name)[0]

    sheet_names = file.sheet_names()
    data = file.sheet_by_name(sheet_names[0])
    for i in range(0, len(data.row(0))):
        if('glucose' in data.row(0)[i].value.lower()):
            index = i + 1
            break
    
    for row in range(0, data.nrows):
        for pattern in eating_string:
            if(not isinstance(data.cell(row, index).value, float) and data.cell(row, index).value.lower().find(pattern) != -1):
                date_values = xlrd.xldate_as_tuple(data.cell(row, 2).value, file.datemode)
                time_value = time(*date_values[3:])
                # id, eating_string, time_elapsed, row (starts from 0)
                mealtime_file.write("{},{},{},{}\n".format(id, pattern, time_value, row))
                

In [6]:
mealtime_file.close()

## Merge and Write User CGMS Readings

In [7]:
cgms_file = open("cgms.csv", "w")

In [8]:
for file_name in file_names:
    file = xlrd.open_workbook('Data/' + file_name)
    id = re.match('T[_,-][0-9]+', file_name)[0]

    for row in range(1, data.nrows):
        if(data.cell(row, 2).value == ''):
            break
        date_values = xlrd.xldate_as_tuple(
            data.cell(row, 2).value, file.datemode)
        time_value = time(*date_values[3:])

        Flow = data.cell(row, 3).value
        CO2_Outlet = data.cell(row, 4).value
        O2_Outlet = data.cell(row, 5).value
        CO2_Inlet = data.cell(row, 6).value
        O2_Inlet = data.cell(row, 7).value
        VCO2 = data.cell(row, 8).value
        VO2 = data.cell(row, 9).value
        RQ = data.cell(row, 10).value
        EE = data.cell(row, 11).value
        Heart_Rate = data.cell(row, 12).value
        Respiratory_Rate = data.cell(row, 13).value
        T_Core = data.cell(row, 14).value
        T_Skin = data.cell(row, 15).value
        Glucose = data.cell(row, 16).value

        cgms_file.write("{},{},{},{}, {},{},{},{}, {},{},{},{}, {},{},{},{}, \n"
                        .format(id, time_value, Flow, CO2_Outlet,
                                O2_Outlet, CO2_Inlet, O2_Inlet, VCO2,
                                VO2, RQ, EE, Heart_Rate,
                                Respiratory_Rate, T_Core, T_Skin, Glucose
                                ))

In [9]:
cgms_file.close()